# GenAI for Humanists
*Capstone project (Felix Aufreiter) - 2024W 136031-1 GenAI for Humanists*

## Abstract/Expose

**ADD TEXT**

## Inicial setup
* Step 1: importing OpenAI API-Key
* Step 2: importing packages
* Step 3: creating a function and test prompt to use **OpenAI API**
* 

In [4]:
!pip install openai python-dotenv

In [21]:
pip install python-docx

Note: you may need to restart the kernel to use updated packages.


In [17]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

# Load environment variables from .env file
_ = load_dotenv(find_dotenv())

# Access your API key
openai.api_key = os.getenv('OPENAI_API_KEY')

In [5]:
#Checking directory for '.env' + '.gitignore'
print("Current working directory:", os.getcwd())
print("Contents:", os.listdir(os.getcwd()))

Current working directory: C:\Users\faufr\Documents\GitHub\GenAI_project
Contents: ['.env', '.git', '.gitignore', '.ipynb_checkpoints', 'capstone_project.ipynb', 'README.md']


In [ ]:
# CHECK your API key
print(os.environ.get("OPENAI_API_KEY"))

## Creating test-prompt

In [9]:

from openai import OpenAI
# Initialize OpenAI client
client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY')
)

def get_completion(prompt, model="gpt-4o-mini"):
    messages = [
        {
            "role": "system",
            "content": "You are a very skilled language teacher. Your answers are precise and you use easy language."
        },
        {
            "role": "user",
            "content": prompt
        }
    ]
    
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content


In [11]:
prompt = "Who is 'The Undertaker' and what are his favorite wrestling moves?"
response = get_completion(prompt)
print(response)

The Undertaker is a famous professional wrestler in WWE (World Wrestling Entertainment). His real name is Mark Calaway, and he is known for his dark, mysterious character and impressive performances in the ring. He has been a part of wrestling for over 30 years and is considered one of the greatest wrestlers of all time.

Some of his favorite wrestling moves include:

1. **Tombstone Piledriver**: This is his signature finishing move, where he lifts his opponent upside down and then drops them on their head.

2. **Chokeslam**: He grabs his opponent by the throat and lifts them high before slamming them down to the mat.

3. **Last Ride**: This is a powerful move where he lifts his opponent onto his shoulders and then slams them down.

4. **Old School**: He walks along the top rope and then jumps down onto his opponent.

These moves are part of what makes The Undertaker a legendary figure in wrestling!


## Importing texts
* example texts to analyze with the LLM

In [25]:
#IMPORTING example text
from docx import Document

# Path to the Word document
file_path = './data/correo electronico.docx'

# Open and read the document
doc = Document(file_path)
print(doc)

# Combine all paragraphs into one string
document_text = "\n".join(paragraph.text for paragraph in doc.paragraphs)
print(document_text)

Correo electronico 

De: adrian123@gmail.com
Para: padres123@gmail.com
Fecha: 12 de novembre de 2024
Asunto: me ciudad favorita

Queridos padres,

Estoy en mi ciuadad favorita, Barcelona. La ciudad es muy bonita y hay mucha gente viviendo aquí. Además, hace mucho calor aquí en Barcelona.

Hoy he visto la ciudad. Para el almuerzo probe tapas en un pequeno restaurante local. ! Fue delicioso! Me gusta de la playa y comer. Luego pasee por el parque y pasee por la playa de la noche. Mas tarde me relaje en la playa de la Barcelona.

Manana tengo planes emocionantes. Quiero visitar muchos mercados y restaurantes para probar mas comida tipica. Tambien espero encontar algunos Souvenirs unicos para llevar a casa.

Estoy disfrutando mucho de mi tiempo aqui y no puedo esperar para contarles mas.

Saludos,
Adrian




## Analyzing the data with an LLM

* The prompt was designed using the CO-STAR framework. (https://towardsdatascience.com/how-i-won-singapores-gpt-4-prompt-engineering-competition-34c195a93d41)
* The aim of this prompt is to provide examples structure for the LLM to correct the provided text sample ("correo_electrónico" --> document_text). The LLM should use the CEFRL (Common Framework of Reference for Language) to set a bar for the expected quality of the written text which was delivered by a student.
* The actual structre of the feedback is derived from the offical Austrian guidelines for grading students' assignements released by the Austrian **Ministry for Education, Science and Research** (

In [29]:
prompt = f"""
Correct the mistakes of the provided text. The text was written by a student. The student should have a language level A2-B1 according
to Common European Framework of Reference for Languages: Learning, Teaching, Assessment CEFRL. Correct the mistakes and provide the adequate solutions.
Don't change the original input text. Only use this structure to correct the text:

Example 1: "Yo te llama José."
Corrected example 1: "Yo te llama José. (Yo me llamo José.)"
Example 2: "Las coches es caros."
Corrected example 2: "Las coches es caros. (Las coches son caros.)"
Example 3: "Hola María, yo te llama José."
Corrected example 3: "Hola María, yo te llama José. (Hola María, me llamo José.) Me gusta los caballo. (Me gustan los caballos.)"

Please correct this text and provide one sentence objective and simple with feedback and which grammar topics the student should improve.
The feedback and explanations have to be in german.
Feedback structure:
"Feedback: [insert feedback sentence here]
An diesem Grammatikthema/diesen Grammatikthemen solltest du noch arbeiten: [insert grammar topics here]
Beurteilungsraster A2:
- Erfüllung der Aufgabenstellung (EA): [insert chosen descriptions]
- Aufbau und Layout (AL): [insert chosen descriptions]
- Spektrum Sprachlicher Mittel (SSM): [insert chosen descriptions]
- Sprachrichtigkeit (SR): [insert chosen descriptions]"
```{document_text}```
"""

response = get_completion(prompt)
print(response)

Correo electronico 

De: adrian123@gmail.com  
Para: padres123@gmail.com  
Fecha: 12 de novembre de 2024  
Asunto: me ciudad favorita  

Queridos padres,  

Estoy en mi ciuadad favorita, Barcelona. (Estoy en mi ciudad favorita, Barcelona.) La ciudad es muy bonita y hay mucha gente viviendo aquí. Además, hace mucho calor aquí en Barcelona.  

Hoy he visto la ciudad. Para el almuerzo probe tapas en un pequeno restaurante local. (Para el almuerzo probé tapas en un pequeño restaurante local.) ! Fue delicioso! Me gusta de la playa y comer. (Me gusta la playa y comer.) Luego pasee por el parque y pasee por la playa de la noche. (Luego paseé por el parque y paseé por la playa de la noche.) Mas tarde me relaje en la playa de la Barcelona. (Más tarde me relajé en la playa de Barcelona.)  

Manana tengo planes emocionantes. (Mañana tengo planes emocionantes.) Quiero visitar muchos mercados y restaurantes para probar mas comida tipica. (Quiero visitar muchos mercados y restaurantes para probar má

## Ad data to the pipeline
* Beurteilungsraster
* templates for text types
* connect to cloud

https://www.reddit.com/r/LangChain/comments/18dloff/attaching_files_to_user_prompt_when_using/?rdt=42513

https://python.langchain.com/docs/tutorials/rag/

https://llamahub.ai/

* python analyzer: https://pypi.org/project/cefrpy/

In [32]:
# Langchain?
# Llamaindex? --> RAG?

# To-do:
* add langchain? (add tools and other methods?)
* add MS Outlook Mail or MS Teams message?
* create a pdf out of the feedback?
* RAG with langchain: https://levelup.gitconnected.com/unlocking-llms-potential-with-rag-a-complete-guide-from-basics-to-advanced-techniques-b4557f268134


# Future use
* connect LLM to SQL-database (CONNECTION to project of RESEARCH SEMINAR): https://medium.com/dataherald/how-to-connect-llm-to-sql-database-with-langchain-sqlagent-48635fddaa74
* connect to database of "Aufgabenpool" tasks: https://aufgabenpool.at/srdp_lfs/index.php?id=sp
* langchain + sqlchain https://medium.com/dataherald/how-to-langchain-sqlchain-c7342dd41614